In [15]:
import torch
import torchaudio
from torch import nn
from torch.nn import functional as F

In [1]:
import sys
sys.path.append('../wave_wizard/')

In [2]:
from src.dataset import get_loader

In [6]:
SR = 22050
SECS = 11
length=SR*SECS
sample_rate=22050

In [7]:
dict(ength=SR*SECS)

{'ength': 242550}

In [13]:
config = dict({
    'dataset': {
        'json_dir': '../dataset/',
        'length': SR*SECS,
        'sample_rate': SR,
    },
    'dataloader': {
        'batch_size': 32,
    }
})

loader = get_loader(config)

In [14]:
for noisy, clean in loader:
    break

In [101]:
def get_encoder_block(in_channels, out_channels, kernel_size=3, **kwargs):
    block = nn.Sequential(
        nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, **kwargs),
        nn.ReLU()
    )
    return block

In [102]:
def get_decoder_block(in_channels, out_channels, kernel_size=3, **kwargs):
    block = nn.Sequential(
        nn.ConvTranspose1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, **kwargs),
        nn.ReLU()
    )
    return block

In [103]:
class GateWave(nn.Module):
    def __init__(self, depth=3, scale=2, hidden=32, kernel_size=):
        super(GateWave, self).__init__()
        in_channels = 1
        out_channels = 1
        encoders = []
        decoders = []
        
        in_ch = in_channels
        for i in range(depth):
            encoder = get_encoder_block(in_ch, hidden, kernel_size=3)
            encoders.append(encoder)
            
            decoder = get_decoder_block(hidden,)
            
            
            
        self.encoder = get_encoder_block(1, 10, 3)
        self.decoder = get_decoder_block(10, 1, 3)
    
    def forward(self, x):
        latent = self.encoder(x)
        output = self.decoder(latent)
        return output

SyntaxError: invalid syntax (234475207.py, line 2)

In [97]:
x = noisy

In [104]:
encoder = get_encoder_block(1, 10, 3)
decoder = get_decoder_block(10, 1, 3)

In [107]:
a = []
for i in range(4):
    ch_in = i
    ch_out = i+1
    a.append(get_encoder_block(ch_in, ch_out))

/Users/timur/opt/anaconda3/envs/dev/lib/python3.9/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [87]:
lstm = nn.LSTM(input_size=10, hidden_size=5, batch_first=True)
gru = nn.GRU(input_size=10, hidden_size=10)

In [88]:
x = encoder(x)

In [89]:
x = x.permute(2, 0, 1)

In [90]:
x.shape

torch.Size([242548, 32, 10])

In [91]:
# out = lstm(x)
out = gru(x)

In [92]:
out[0].shape

torch.Size([242548, 32, 10])

In [16]:
## Denoising Convolutional Auto-Encoder
import torch.nn as nn
from torch.nn import functional as F


class SimpleDCAE(nn.Module):
    def __init__(self, s = 1, d = 1):
         super(SimpleDCAE, self).__init__()
         ## encoder: 1 -> 128 -> 64 -> 32
         self.encoder = nn.ModuleDict({
             'enc_block1': self.create_encoder_block(1, 128, stride = s, dilation =d, padding = 0 ),
             'enc_block2': self.create_encoder_block(128, 64, stride = s, dilation =d, padding = 0),
              'enc_block3': self.create_encoder_block(64, 32, stride = s, dilation =d, padding = 0) 
            })
         
         ## decoder: 32 -> 64 -> 128 -> 1
         self.decoder =  nn.ModuleDict(
             {
              'dec_block1': self.create_decoder_block(32, 64, stride = s, dilation = d, padding = 0),
              'dec_block2' : self.create_decoder_block(64, 128, stride = s, dilation = d, padding = 0),
              'dec_block3': nn.ConvTranspose2d(in_channels=128, out_channels=1,
                                               dilation = d,
                                               kernel_size=3,stride = s, padding = 0)
             }
              )
    def create_encoder_block(self, in_channels, out_channels, **kwargs):
        block = nn.ModuleList([nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, **kwargs),
                nn.BatchNorm2d(out_channels),
                nn.LeakyReLU()
                ])
        return nn.Sequential(*block)
    
    def create_decoder_block(self, in_channels, out_channels, **kwargs):
        block = nn.ModuleList([
            nn.ConvTranspose2d(in_channels=in_channels,
                               out_channels=out_channels,
                               kernel_size=3, **kwargs),
                nn.LeakyReLU()
                ])
        return nn.Sequential(*block)
    
    def forward(self, x):
        ## encoder layer
        for e in self.encoder:
            x = self.encoder[e](x)
        
        ## decoder layer
        for d in self.decoder:
            x = self.decoder[d](x)
            
        return 